In [37]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [39]:
# Read csv file with results from the simulation - Bike tasks

#Import files
os.chdir("/Users/naroacorettisanchez/Documents/GitHub/MultifunctionalVehicleFleets/results/38reps_MIXED_fullData_final/")
extension = 'csv'

#Bike tasks
filenames =[i for i in glob.glob('autonomousBike_trip_event*.{}'.format(extension))]

#Create a list with the dataframes in filenames
dfs = []
for filename in filenames:
    df_tasks = pd.read_csv(filename)
    dfs.append(df_tasks)


In [50]:
# Create a new list to store the modified dataframes
dfs_modified = []

# Iterate over each dataframe in dfs
for df_tasks in dfs:
    # Select the desired columns
    df_modified = df_tasks[['Cycle', 'Day', 'Time', 'Agent', 'Message']]
    # Append the modified dataframe to the new list
    dfs_modified.append(df_modified)


dfs_modified[0].head()

,Cycle,Day,Time,Agent,Message
0,0,7,00:00:00,autonomousBike0,nil
1,0,7,00:00:00,autonomousBike1,nil
2,0,7,00:00:00,autonomousBike2,nil
3,0,7,00:00:00,autonomousBike3,nil
4,0,7,00:00:00,autonomousBike4,nil


In [45]:
# for df in dfs_modified: Keep only the rows with 'Message' starting by 'In Use*'
for df in dfs_modified:
    df.drop(df[~df['Message'].str.startswith('In Use')].index, inplace = True)

dfs_modified[0].head()

/Users/naroacorettisanchez/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Cycle,Day,Time,Agent,Message
342,770,7,01:04:10,autonomousBike293,In Use people(3471)
344,774,7,01:04:30,autonomousBike148,In Use people(3472)
348,836,7,01:09:40,autonomousBike319,In Use people(3473)
354,900,7,01:15:00,autonomousBike166,In Use package(0)
358,919,7,01:16:35,autonomousBike4,In Use people(3474)


In [46]:
# Crop Message column string to get only the task name

for df in dfs_modified:
    df['Message'] = df['Message'].str.split('(').str[0]

dfs_modified[0].head()


/Users/naroacorettisanchez/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Cycle,Day,Time,Agent,Message
342,770,7,01:04:10,autonomousBike293,In Use people
344,774,7,01:04:30,autonomousBike148,In Use people
348,836,7,01:09:40,autonomousBike319,In Use people
354,900,7,01:15:00,autonomousBike166,In Use package
358,919,7,01:16:35,autonomousBike4,In Use people


In [48]:
# Create a new list to store the modified dataframes
dfs_prev_message = []

for df_tasks_use in dfs_modified:
    # Reorder rows to get all the events for each agent together
    df_tasks_use = df_tasks_use.sort_values(by=['Agent', 'Day', 'Time'])
    #Add a column with the previous message, in case it is from the same agent
    df_tasks_use['Previous_Message'] = df_tasks_use['Message'].shift().where(df_tasks_use['Agent'].eq(df_tasks_use['Agent'].shift()), np.nan)
    dfs_prev_message.append(df_tasks_use)

dfs_prev_message[0].head()

,Cycle,Day,Time,Agent,Message,Previous_Message
3296,6123,7,08:30:15,autonomousBike0,In Use package,NaN
5080,6565,7,09:07:05,autonomousBike0,In Use people,In Use package
6620,7055,7,09:47:55,autonomousBike0,In Use people,In Use people
7192,7199,7,09:59:55,autonomousBike0,In Use package,In Use people
8599,7592,7,10:32:40,autonomousBike0,In Use package,In Use package


In [51]:
for df_tasks_use in dfs_prev_message:
    # For each row, check if Message is the same as Previous_Message, if yes add True to the column 'Same'
    df_tasks_use['Same'] = np.where(df_tasks_use['Message'] == df_tasks_use['Previous_Message'], True, False)
    # For each row, if Previous Message is NaN, add NaN to the column 'Same'
    df_tasks_use['Same'] = np.where(df_tasks_use['Previous_Message'].isnull(), np.nan, df_tasks_use['Same'])
    
dfs_prev_message[0].head()

,Cycle,Day,Time,Agent,Message,Previous_Message,Same
3296,6123,7,08:30:15,autonomousBike0,In Use package,NaN,NaN
5080,6565,7,09:07:05,autonomousBike0,In Use people,In Use package,0.0
6620,7055,7,09:47:55,autonomousBike0,In Use people,In Use people,1.0
7192,7199,7,09:59:55,autonomousBike0,In Use package,In Use people,0.0
8599,7592,7,10:32:40,autonomousBike0,In Use package,In Use package,1.0


In [53]:
average_counts = []
for df_tasks_use in dfs_prev_message:
    counts = df_tasks_use['Same'].value_counts(normalize=True) * 100
    average_counts.append(counts)

average_values = pd.concat(average_counts).groupby(level=0).mean()
print(average_values)


0.0    36.026476
1.0    63.973524
Name: Same, dtype: float64
